# SF-DAT-21 | Unit Project 3

In this project, you will perform a logistic regression on the admissions data we've been working with in Unit Projects 1 and 2.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
import pylab as pl
%matplotlib inline

In [2]:
df_raw = pd.read_csv("../../dataset/admissions.csv")
df = df_raw.dropna()
print df.shape
print df.head()

(397, 4)
   admit  gre   gpa  prestige
0      0  380  3.61         3
1      1  660  3.67         3
2      1  800  4.00         1
3      1  640  3.19         4
4      0  520  2.93         4


## Part 1. Frequency Tables

#### Question 1. Let's create a frequency table of our variables.

In [5]:
# frequency table for prestige and whether or not someone was admitted
freq = pd.crosstab(index=df['prestige'],columns=df['admit'])
freq 

admit,0,1
prestige,,
1,28,33
2,95,53
3,93,28
4,55,12


## Part 2. Return of dummy variables

#### Question 2.1. Create class or dummy variables for prestige.

In [6]:
df.head(2)

,admit,gre,gpa,prestige
0,0,380,3.61,3
1,1,660,3.67,3


In [7]:
df.prestige.value_counts()

2    148
3    121
4     67
1     61
Name: prestige, dtype: int64

In [10]:
dummy_ranks = pd.get_dummies(df['prestige'],prefix='prestige')
dummy_ranks.head(2)

,prestige_1.0,prestige_2.0,prestige_3.0,prestige_4.0
0,0,0,1,0
1,0,0,1,0


#### Question 2.2. When modeling our class variables, how many do we need?

Answer: 3

## Part 3. Hand calculating odds ratios

Develop your intuition about expected outcomes by hand calculating odds ratios.

In [11]:
cols_to_keep = ['admit', 'gre', 'gpa']
handCalc = df[cols_to_keep].join(dummy_ranks.ix[:, 'prestige_1':])
print handCalc.head()

   admit  gre   gpa  prestige_1.0  prestige_2.0  prestige_3.0  prestige_4.0
0      0  380  3.61             0             0             1             0
1      1  660  3.67             0             0             1             0
2      1  800  4.00             1             0             0             0
3      1  640  3.19             0             0             0             1
4      0  520  2.93             0             0             0             1


In [27]:
# crosstab prestige 1 admission
# frequency table cutting prestige and whether or not someone was admitted
freq = pd.crosstab(index=handCalc['prestige_1.0'],columns=handCalc['admit'], margins=True)
freq.columns = ['admit_yes','not_admit_no','rowtotal']
freq.index = ['prestige_0.0','prestige_1.0','columntotal']
freq

,admit_yes,not_admit_no,rowtotal
prestige_0.0,243,93,336
prestige_1.0,28,33,61
columntotal,271,126,397


#### Question 3.1. Use the cross tab above to calculate the odds of being admitted to grad school if you attended a #1 ranked college.

In [17]:
odds_admitted_1 = (1.*28/61)/(1.* 33/61)
odds_admitted_1

0.8484848484848484

#### Question 3.2. Now calculate the odds of admission if you did not attend a #1 ranked college.

In [28]:
odds_admitted_no1 = (1. * 243)/(1.*336)
odds_admitted_no1

0.7232142857142857

#### Question 3.3. Calculate the odds ratio.

In [20]:
odds_ratio = odds_admitted_1/odds_admitted_no1
odds_ratio

1.173213617658062

#### Question 3.4. Write this finding in a sentenance:

Answer: so odds of admission are about .17 greater than for students attended #1 ranked college

#### Question 3.5. Print the cross tab for prestige_4.

In [30]:
# crosstab prestige 1 admission
# frequency table cutting prestige and whether or not someone was admitted
freq2 = pd.crosstab(index=handCalc['prestige_4.0'],columns=handCalc['admit'], margins=True)
freq2.columns = ['admit_yes','not_admit_no','rowtotal']
freq2.index = ['prestige_0.0','prestige_4.0','columntotal']
freq2

,admit_yes,not_admit_no,rowtotal
prestige_0.0,216,114,330
prestige_4.0,55,12,67
columntotal,271,126,397


#### Question 3.6. Calculate the OR.

In [24]:
odds_admitted_4 = (1.*55/67)/(1.* 12/67)
odds_admitted_4
odds_admitted_no4 = (1. * (271 - 55))/(1.*(397 - 67))
odds_admitted_no4
odds_ratio = odds_admitted_4/odds_admitted_no4
odds_ratio

7.002314814814816

#### Question 3.7. Write this finding in a sentence.

Answer: so odds of admission are about 6 greater than for students attended #4 ranked college

## Part 4. Analysis

In [31]:
# create a clean data frame for the regression
cols_to_keep = ['admit', 'gre', 'gpa']
data = df[cols_to_keep].join(dummy_ranks.ix[:, 'prestige_2':])
print data.head()

   admit  gre   gpa  prestige_2.0  prestige_3.0  prestige_4.0
0      0  380  3.61             0             1             0
1      1  660  3.67             0             1             0
2      1  800  4.00             0             0             0
3      1  640  3.19             0             0             1
4      0  520  2.93             0             0             1


We're going to add a constant term for our Logistic Regression.  The statsmodels function we're going to be using requires that intercepts/constants are specified explicitly.

In [32]:
# manually add the intercept
data['intercept'] = 1.0

In [33]:
data.head(3)

,admit,gre,gpa,prestige_2.0,prestige_3.0,prestige_4.0,intercept
0,0,380,3.61,0,1,0,1
1,1,660,3.67,0,1,0,1
2,1,800,4.00,0,0,0,1


#### Question 4.1. Set the covariates to a variable called train_cols.

In [34]:
train_cols = ['intercept','gre','gpa','prestige_2.0','prestige_3.0','prestige_4.0']

#### Question 4.2. Fit the model.

In [35]:
from sklearn import linear_model

X = data[train_cols]
y = data['admit']

model = linear_model.LogisticRegression()
model.fit(X,y)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

#### Question 4.3. Print the summary results.

In [36]:
model.coef_

array([[-1.09518905,  0.00188666,  0.32452811, -0.6200671 , -1.16988125,
        -1.3847737 ]])

#### Question 4.4. Calculate the odds ratios of the coeffincients and their 95% CI intervals

hint 1: np.exp(X)

hint 2: conf['OR'] = params

        conf.columns = ['2.5%', '97.5%', 'OR']

In [39]:
for (a,b) in zip(train_cols, np.exp(model.coef_[0]) - 1):
    print '{:20}{}'.format(a,b)

intercept           -0.665523632491
gre                 0.00188844039508
gpa                 0.383377688876
prestige_2.0        -0.46209165672
prestige_3.0        -0.689596200537
prestige_4.0        -0.749619545953


In [40]:
model.conf_

AttributeError: 'LogisticRegression' object has no attribute 'conf_'

#### Question 4.5. Interpret the OR of Prestige_2.

Answer:

#### Question 4.6. Interpret the OR of GPA.

Answer:

## Part 5: Predicted probablities


As a way of evaluating our classifier, we're going to recreate the dataset with every logical combination of input values.  This will allow us to see how the predicted probability of admission increases/decreases across different variables.  First we're going to generate the combinations using a helper function called cartesian (above).

We're going to use np.linspace to create a range of values for "gre" and "gpa".  This creates a range of linearly spaced values from a specified min and maximum value--in our case just the min/max observed values.

In [ ]:
def cartesian(arrays, out=None):
    """
    Generate a cartesian product of input arrays.
    Parameters
    ----------
    arrays : list of array-like
        1-D arrays to form the cartesian product of.
    out : ndarray
        Array to place the cartesian product in.
    Returns
    -------
    out : ndarray
        2-D array of shape (M, len(arrays)) containing cartesian products
        formed of input arrays.
    Examples
    --------
    >>> cartesian(([1, 2, 3], [4, 5], [6, 7]))
    array([[1, 4, 6],
           [1, 4, 7],
           [1, 5, 6],
           [1, 5, 7],
           [2, 4, 6],
           [2, 4, 7],
           [2, 5, 6],
           [2, 5, 7],
           [3, 4, 6],
           [3, 4, 7],
           [3, 5, 6],
           [3, 5, 7]])
    """

    arrays = [np.asarray(x) for x in arrays]
    dtype = arrays[0].dtype

    n = np.prod([x.size for x in arrays])
    if out is None:
        out = np.zeros([n, len(arrays)], dtype=dtype)

    m = n / arrays[0].size
    out[:,0] = np.repeat(arrays[0], m)
    if arrays[1:]:
        cartesian(arrays[1:], out=out[0:m,1:])
        for j in xrange(1, arrays[0].size):
            out[j*m:(j+1)*m,1:] = out[0:m,1:]
    return out

In [ ]:
# instead of generating all possible values of GRE and GPA, we're going
# to use an evenly spaced range of 10 values from the min to the max
gres = np.linspace(data['gre'].min(), data['gre'].max(), 10)

print gres
# array([ 220.        ,  284.44444444,  348.88888889,  413.33333333,
#         477.77777778,  542.22222222,  606.66666667,  671.11111111,
#         735.55555556,  800.        ])

gpas = np.linspace(data['gpa'].min(), data['gpa'].max(), 10)

print gpas
# array([ 2.26      ,  2.45333333,  2.64666667,  2.84      ,  3.03333333,
#         3.22666667,  3.42      ,  3.61333333,  3.80666667,  4.        ])

# enumerate all possibilities
combos = pd.DataFrame(cartesian([gres, gpas, [1, 2, 3, 4], [1.]]))

#### Question 5.1. Recreate the dummy variables.

In [ ]:
# recreate the dummy variables

# keep only what we need for making predictions


#### Question 5.2. Make predictions on the enumerated dataset.

#### Question 5.3. Interpret findings for the last 4 observations.

Answer:

## Bonus

Plot the probability of being admitted into graduate school, stratified by GPA and GRE score.